In [1]:
%load_ext autoreload
%autoreload 2

from pymongo import ASCENDING, GEOSPHERE, MongoClient
import pandas as pd
from alive_progress import alive_bar
from shapely.geometry import Point, mapping
from keplergl import KeplerGl
import shapely
import json
from os import listdir
from os.path import isfile, join
from tqdm import tqdm
import geopandas as gpd
from h3 import h3
import math

In [2]:
client = MongoClient('mongodb://localhost:27017/')

db = client.osmDataDB
coll_areas = db.areas
coll_relations = db.relations
coll_nextBikeRaw = db.nextBikeRaw
coll_cities = db.cities
coll_stations = db.stationsInCities

In [3]:
RESOLUTIONS = [9, 10, 11]

In [4]:
NEIGHBORS = {}
for res in RESOLUTIONS:
    edge_size = h3.edge_length(res, unit='m')
    h = edge_size * math.sqrt(3) / 2
    k_neighbours = math.ceil(2000 / (h*2))
    print(res, edge_size, h, k_neighbours)
    NEIGHBORS[res] = k_neighbours

9 174.37566800000002 151.01375828988122 7
10 65.907807 57.07783516972185 18
11 24.910561 21.57317864852189 47


In [5]:
def getHexes(city_id, resolution):
    station_indexes = set()
    all_indexes = set()
    for st in coll_stations.find({ 'city_id': city_id }):
        pt = shapely.geometry.shape(st['geometry'])
        h = h3.geo_to_h3(pt.x, pt.y, resolution)
        indexes = h3.hex_range(h, NEIGHBORS[resolution])
        station_indexes.add(h)
        all_indexes.update(indexes)
    return station_indexes, all_indexes

In [6]:
hexes_dict = {}

In [7]:
for city in coll_cities.find({ 'accepted': True }):
    print(city['city'])
    hexes_dict[city['city']] = {}
    for resolution in RESOLUTIONS:
        hexes_dict[city['city']][resolution] = {}
        stations, all_hexes = getHexes(city['city_id'], resolution)
        hexes_dict[city['city']][resolution]['stations'] = stations
        hexes_dict[city['city']][resolution]['all_hexes'] = all_hexes

Antwerpen
Barcelona
Berlin
Bern
Bordeaux
Brno
Bruxelles
Budapest
Cardiff
Dublin
Gothenburg
Helsinki
Kyiv
London
Lyon
Madrid
Marseille
Milan
Moscow
Munich
Nantes
Oslo
Ostrava
Paris
Poznań
Prague
Seville
Toulouse
Valencia
Vienna
Warszawa
Wrocław
Zaragoza
Zurich


In [11]:
def get_parent_city_id(polygon_hex):
    city = coll_cities.find_one({
        "geometry": {
            "$geoIntersects": {
                "$geometry": shapely.geometry.mapping(polygon_hex)
            }
        }
    })
    city_id = None
    if city is not None:
        city_id = city['city_id']
        # print(city['city'])
    return city_id

In [23]:
def is_in_city(polygon_hex, city_id):
    city = coll_cities.find_one({
        "city_id": city_id,
        "geometry": {
            "$geoIntersects": {
                "$geometry": shapely.geometry.mapping(polygon_hex)
            }
        }
    })
    return city is not None

In [9]:
current_res = 9

In [13]:
polylines = []
hexes = [h for dct in hexes_dict.values() for h in dct[current_res]['all_hexes']]
for hex in tqdm(hexes):
    polygons = h3.h3_set_to_multi_polygon([hex], geo_json=False)
    # flatten polygons into loops.
    outlines = [loop for polygon in polygons for loop in polygon]
    polyline = [outline + [outline[0]] for outline in outlines][0]
    polygon = shapely.geometry.Polygon(polyline)
    city_id = get_parent_city_id(polygon)
    if city_id is not None:
        polylines.append(polygon)
h3_gdf = gpd.GeoDataFrame({'geometry': polylines}, crs="EPSG:3857")

polylines = []
stations = [h for dct in hexes_dict.values() for h in dct[current_res]['stations']]
for hex in tqdm(stations):
    polygons = h3.h3_set_to_multi_polygon([hex], geo_json=False)
    # flatten polygons into loops.
    outlines = [loop for polygon in polygons for loop in polygon]
    polyline = [outline + [outline[0]] for outline in outlines][0]
    polygon = shapely.geometry.Polygon(polyline)
    city_id = get_parent_city_id(polygon)
    if city_id is not None:
        polylines.append(polygon)
h3_gdf_2 = gpd.GeoDataFrame({'geometry': polylines}, crs="EPSG:3857")

100%|██████████| 9304/9304 [02:35<00:00, 59.95it/s] 


In [4]:
coll_hexes = db.hexesInCities
coll_hexes.create_index([("city_id", ASCENDING)])
coll_hexes.create_index([("hex_id", ASCENDING)])
coll_hexes.create_index([("geometry", GEOSPHERE)])

'geometry_2dsphere'

In [29]:
for res in RESOLUTIONS:
    hexes = [h for dct in hexes_dict.values() for h in dct[res]['all_hexes']]
    stations = [h for dct in hexes_dict.values() for h in dct[res]['stations']]
    for hex in tqdm(hexes, desc=f"Resolution: {res}"):
        has_station = hex in stations
        polygons = h3.h3_set_to_multi_polygon([hex], geo_json=False)
        outlines = [loop for polygon in polygons for loop in polygon]
        polyline = [outline + [outline[0]] for outline in outlines][0]
        polygon = shapely.geometry.Polygon(polyline)
        city_id = get_parent_city_id(polygon)
        if city_id is not None:
            coll_hexes.insert_one({
                'hex_id': hex,
                'city_id': city_id,
                'resolution': res,
                'has_station': has_station,
                'geometry': shapely.geometry.mapping(polygon)
            })


Resolution: 11: 100%|██████████| 5137395/5137395 [21:16:07<00:00, 67.10it/s] 


In [5]:
coll_hexes_filtered = db.hexesInCitiesFiltered
coll_hexes_filtered.create_index([("city_id", ASCENDING)])
coll_hexes_filtered.create_index([("hex_id", ASCENDING)])
coll_hexes_filtered.create_index([("geometry", GEOSPHERE)])

'geometry_2dsphere'

In [6]:
coll_relations_filtered = db.relationsFiltered
coll_relations_filtered.create_index([("city_id", ASCENDING)])
coll_relations_filtered.create_index([("osm_id", ASCENDING)])
# coll_relations_filtered.create_index([("hex_id", ASCENDING)])
coll_relations_filtered.create_index([("geometry", GEOSPHERE)])

'geometry_2dsphere'

In [18]:
import multiprocessing
from pymongo import MongoClient
from functools import partial
from itertools import chain
import time
from tqdm.contrib.concurrent import process_map

def chunks(l, n):
    for i in range(0, len(l), n):
        yield l[i:i + n]

def parse_hex(chunk):
    #define client inside function
    client = client = MongoClient('localhost',27017,maxPoolSize=10000)
    db = client.osmDataDB
    coll_hexes = db.hexesInCities
    coll_relations = db.relations
    coll_hexes_filtered = db.hexesInCitiesFiltered
    coll_relations_filtered = db.relationsFiltered
    for _id in chunk:
        hex = coll_hexes.find_one({ '_id': _id })
        if coll_hexes_filtered.find_one({ 'hex_id': hex['hex_id'] }) is None:
            relations = list(coll_relations.find({
                "geometry": {
                    "$geoIntersects": {
                        "$geometry": hex['geometry']
                    }
                }
            }))
            if len(relations) > 0:
                relations_to_add = []
                for relation in relations:
                    if coll_relations_filtered.find_one({ 'osm_id': relation['osm_id'] }) is None:
                        relation['city_id'] = hex['city_id']
                        del relation['_id']
                        relations_to_add.append(relation)
                if len(relations_to_add) > 0:
                    coll_relations_filtered.insert_many(relations_to_add)
                del hex['_id']
                coll_hexes_filtered.insert_one(hex)

In [16]:
result = coll_hexes.find({"resolution": { "$gt": 9 }},{"_id":1})
hex_ids = [x["_id"] for x in result]
del result
chunks_list = list(chunks(hex_ids,100))
del hex_ids

In [19]:
process_map(parse_hex, chunks_list, max_workers=8)

<ipython-input-19-75d20c6237a4>:1: TqdmWarning: Iterable length 55840 > 1000 but `chunksize` is not set. This may seriously degrade multiprocess performance. Set `chunksize=1` or more.
  process_map(parse_hex, chunks_list, max_workers=8)
  0%|          | 180/55840 [01:00<5:12:54,  2.96it/s] 
Exception ignored in: Exception ignored in: Exception ignored in: <function WeakKeyDictionary.__init__.<locals>.remove at 0x7f241010e550><function WeakKeyDictionary.__init__.<locals>.remove at 0x7f241010e550><function WeakKeyDictionary.__init__.<locals>.remove at 0x7f241010e550>


Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/raczeq/anaconda3/lib/python3.8/weakref.py", line 345, in remove
  File "/home/raczeq/anaconda3/lib/python3.8/weakref.py", line 345, in remove
  File "/home/raczeq/anaconda3/lib/python3.8/weakref.py", line 345, in remove
    def remove(k, selfref=ref(self)):
KeyboardInterrupt        def remove(k, selfref=r

In [14]:
source_json = h3_gdf.to_json()
with open(f'all_hexes_{current_res}.geojson', 'w', encoding='utf-8') as source:
    source.write(source_json)

source_json = h3_gdf_2.to_json()
with open(f'station_hexes_{current_res}.geojson', 'w', encoding='utf-8') as source:
    source.write(source_json)

In [14]:
map_1 = KeplerGl(height=800)
map_1.add_data(data=h3_gdf, name='polylines')
map_1.add_data(data=h3_gdf_2, name='stations')
map_1

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'polylines': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 2…

In [22]:
def save_data(collection, filter, file_name):
    records = [r for r in tqdm(collection.find(filter))]
    features = []
    for r in records:
        # r['geometry'] = shapely.geometry.shape(r['geometry'])
        geom = r['geometry']
        del r['_id']
        del r['geometry']
        features.append({
            "type": "Feature",
            "geometry": geom,
            "properties": r
        })
    full_dict = {
        "type": "FeatureCollection",
        "features": features
    }
    # df = gpd.GeoDataFrame(records)
    df = gpd.GeoDataFrame.from_features(full_dict)
    print(df)
    # df.to_file(file_name, driver='GeoJSON')
    try:
        source_json = df.to_json(na='drop')
    except:
        source_json = df.to_json()
    with open(file_name, 'w', encoding='utf-8') as source:
        source.write(source_json)

In [18]:
save_data(coll_relations_filtered, {'city_id': 30}, 'london_relations.geojson')
save_data(coll_hexes_filtered, {'city_id': 30, 'resolution': 9 }, 'london_hexes_9.geojson')
save_data(coll_hexes_filtered, {'city_id': 30, 'resolution': 10 }, 'london_hexes_10.geojson')
save_data(coll_hexes_filtered, {'city_id': 30, 'resolution': 11 }, 'london_hexes_11.geojson')
# save_data()

153340it [00:03, 41061.17it/s]
                                                 geometry  type      highway  \
0       LINESTRING (-0.08605 51.51087, -0.08621 51.51083)   way      primary   
1       LINESTRING (-0.00140 51.47808, -0.00148 51.478...   way      footway   
2                               POINT (-0.07376 51.55081)  node          NaN   
3       LINESTRING (-0.04586 51.54384, -0.04588 51.543...   way  residential   
4                               POINT (-0.00060 51.55666)  node          NaN   
...                                                   ...   ...          ...   
153335  LINESTRING (-0.06134 51.52942, -0.06155 51.52938)   way      service   
153336  LINESTRING (-0.06072 51.52938, -0.06073 51.52939)   way      footway   
153337                          POINT (-0.06119 51.52934)  node          NaN   
153338                          POINT (-0.06116 51.52935)  node          NaN   
153339                          POINT (-0.06113 51.52936)  node          NaN   

        

In [23]:
save_data(coll_relations_filtered, {'city_id': 59}, 'wroclaw_relations.geojson')
save_data(coll_hexes_filtered, {'city_id': 59, 'resolution': 9 }, 'wroclaw_hexes_9.geojson')
save_data(coll_hexes_filtered, {'city_id': 59, 'resolution': 10 }, 'wroclaw_hexes_10.geojson')
save_data(coll_hexes_filtered, {'city_id': 59, 'resolution': 11 }, 'wroclaw_hexes_11.geojson')

97232it [00:02, 42110.13it/s]
                                                geometry  type leisure  sport  \
0      POLYGON ((17.07156 51.14777, 17.07187 51.14765...   way   pitch  multi   
1      LINESTRING (17.13018 51.15531, 17.13086 51.154...   way     NaN    NaN   
2      LINESTRING (17.00134 51.18097, 17.00129 51.180...   way     NaN    NaN   
3      LINESTRING (16.96709 51.13757, 16.96755 51.13757)   way     NaN    NaN   
4                              POINT (17.03451 51.09924)  node     NaN    NaN   
...                                                  ...   ...     ...    ...   
97227  POLYGON ((17.00254 51.12034, 17.00232 51.11997...   way   pitch  multi   
97228                          POINT (17.00227 51.12105)  node     NaN    NaN   
97229                          POINT (17.00220 51.12098)  node     NaN    NaN   
97230  LINESTRING (17.00205 51.12113, 17.00192 51.12152)   way     NaN    NaN   
97231  LINESTRING (17.00228 51.12115, 17.00233 51.12100)   way     NaN    NaN  